In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("seminar-rs-token")

# !git clone https://{token}@github.com/user/repo.git
!git clone https://{token}@github.com/JebediahC/seminar-rs.git

!ls seminar-rs/

In [ ]:
%cd seminar-rs/

# download sub modules
!git submodule update --init --recursive
%cd UniMatch-V2/
!ls

In [ ]:
# download pretrained models
!mkdir -p pretrained_models
!wget -P pretrained_models https://dl.fbaipublicfiles.com/dinov2/dinov2_vits14/dinov2_vits14_pretrain.pth
!ls pretrained_models/

In [1]:
# Link pretrained model to expected location
!mkdir -p pretrained
!ln -s ../pretrained_models/dinov2_vits14_pretrain.pth pretrained/dinov2_small.pth
!ls -lh pretrained/

total 0
lrwxrwxrwx 1 jebediahc jebediahc 47 Dec 14 20:07 dinov2_small.pth -> ../pretrained_models/dinov2_vits14_pretrain.pth


In [ ]:
# preprocess datasets
!mkdir -p coco-2017
!ln -s /kaggle/input/coco-2017-dataset/coco2017/train2017 coco-2017/train2017
!ln -s /kaggle/input/coco-2017-dataset/coco2017/val2017 coco-2017/val2017
!ln -s /kaggle/input/coco-2017-dataset/coco2017/annotations coco-2017/annotations
!ln -s /kaggle/input/coco-2017-masks-for-unimatch-v2/masks coco-2017/masks

In [ ]:

train_script = """#!/bin/bash

# Custom configuration for single GPU training
dataset='coco'
method='unimatch_v2'
exp='dinov2_small'
split='1_32'

config=configs/${dataset}.yaml
labeled_id_path=splits/$dataset/$split/labeled.txt
unlabeled_id_path=splits/$dataset/$split/unlabeled.txt
save_path=exp/$dataset/$method/$exp/$split

mkdir -p $save_path

python -m torch.distributed.launch \
    --nproc_per_node=$1 \
    --master_addr=localhost \
    --master_port=$2 \
    $method.py \
    --config=$config --labeled-id-path $labeled_id_path --unlabeled-id-path $unlabeled_id_path \
    --save-path $save_path --port $2 2>&1 | tee $save_path/out.log
"""

with open('scripts/train_custom.sh', 'w') as f:
    f.write(train_script)

!chmod +x scripts/train_custom.sh
!cat scripts/train_custom.sh

In [ ]:
# use python read configs/coco.yaml and change data_root to point to coco-2017
# Also reduce batch_size and crop_size to save GPU memory
import yaml
with open('configs/coco.yaml', 'r') as f:
    config = yaml.safe_load(f)
config['data_root'] = 'coco-2017'
# Reduce memory consumption
config['batch_size'] = 2  # Reduced from 4
# config['crop_size'] = 384  # Reduced from 518
with open('configs/coco.yaml', 'w') as f:
    yaml.dump(config, f)

!cat configs/coco.yaml

In [ ]:
# Single GPU training: pass 1 for nproc_per_node and 29500 for port
!sh scripts/train_custom.sh 1 29500